In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [3]:
#依赖的包
from lib.dataloader import normal_and_generate_dataset_time, get_mask, get_adjacent, get_grid_node_map_maxtrix
#攻击函数import
from lib.utils_new import mask_loss, compute_loss, predict_and_evaluate, attack, random_attack, fgsm_attack, min_attack
from lib.utils_new import saliency, saliency_loss, attack_js, attack_js,attack_saliency,rand_attack
from model.GSNet import GSNet
from lib.early_stop import EarlyStopping
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import json
import configparser
import pickle as pkl
from time import time
from datetime import datetime
import shutil
import argparse
import random
import math

import sys
import os


/home/wyb/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/wyb/miniconda3/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [4]:
#完整运行时会生成txt日志记录#
train_log_filename = "test_log.txt"
train_log_filepath = os.path.join("./", train_log_filename)
now = datetime.now()
date_time = now.strftime("|%Y-%m-%d, %H:%M:%S| ")

#以命令方式运行时会自动设置工作空间
#file会报错，应该家’‘
curPath = os.path.abspath(os.path.dirname('__file__'))
rootPath = os.path.split(curPath)[0]
sys.path.append(rootPath)

In [5]:
#配置文件载入，初始参数记录

config_filename = "config/nyc/GSNet_NYC_Config.json"
with open(config_filename, 'r') as f:
    config = json.loads(f.read())
# 写入测试超参数
with open(train_log_filepath, "a") as f:
    f.write('测试文件运行时间：'+date_time)
    f.write('\r\n')
    f.write('超参数设置：')
    f.write('\r\n')
    f.write(json.dumps(config, sort_keys=True, indent=4))
    f.write('\r\n')
f.close
#GPU设置
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())

#文件读取
#命令行参数实质上就是赋初始值的作用，可以删除直接赋值就好
north_south_map = config['north_south_map']
west_east_map = config['west_east_map']

all_data_filename = config['all_data_filename']
mask_filename = config['mask_filename']

road_adj_filename = config['road_adj_filename']
risk_adj_filename = config['risk_adj_filename']
poi_adj_filename = config['poi_adj_filename']
grid_node_filename = config['grid_node_filename']
grid_node_map = get_grid_node_map_maxtrix(grid_node_filename)
num_of_vertices = grid_node_map.shape[1]


patience = config['patience']#用处：
delta = config['delta']

if config['seed'] is not None:
    seed = config['seed']
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    np.random.seed(seed)
    random.seed(seed)


train_rate = config['train_rate']
valid_rate = config['valid_rate']

recent_prior = config['recent_prior']
week_prior = config['week_prior']
one_day_period = config['one_day_period']
days_of_week = config['days_of_week']
pre_len = config['pre_len']
seq_len = recent_prior + week_prior

training_epoch = config['training_epoch']


True


# 原始数据处理
all_data.pkl       (8760, 48, 20, 20) 8760=365*24
grid_node_map.pkl  (400, 243)  映射矩阵
输出特征大小
训练集：4584*7*48*20*20
验证集：1080*7*48*20*20
测试集：1080*7*48*20*20
7的含义：t =673时刻是，输入7个时间片，[1, 169, 337, 505, 670, 671, 672]

In [6]:
batch_size = config['batch_size']
learning_rate = config['learning_rate']

#模型结构
num_of_gru_layers = config['num_of_gru_layers']
gru_hidden_size = config['gru_hidden_size']
gcn_num_filter = config['gcn_num_filter']

loaders = []
scaler = ""
train_data_shape = ""
graph_feature_shape = ""
   ################## 获取测试数据#########################
for idx, (x, y, target_times, high_x, high_y, high_target_times, scaler) in enumerate(normal_and_generate_dataset_time(
        all_data_filename,
        train_rate=train_rate,
        valid_rate=valid_rate,
        recent_prior=recent_prior,
        week_prior=week_prior,
        one_day_period=one_day_period,
        days_of_week=days_of_week,
        pre_len=pre_len)):
    if False:#前100个，没写默认是false
        x = x[:100]
        y = y[:100]
        target_times = target_times[:100]
        high_x = high_x[:100]
        high_y = high_y[:100]
        high_target_times = high_target_times[:100]

    if 'nyc' in all_data_filename:
        graph_x = x[:, :, [0, 46, 47], :, :].reshape(
            (x.shape[0], x.shape[1], -1, north_south_map*west_east_map))#4584*7*3*400,注意reshape和交换维度的不同
        high_graph_x = high_x[:, :, [0, 46, 47], :, :].reshape(
            (high_x.shape[0], high_x.shape[1], -1, north_south_map*west_east_map))#1337*7*3*400
        graph_x = np.dot(graph_x, grid_node_map)#4584*7*3*243
        high_graph_x = np.dot(high_graph_x, grid_node_map)
    if 'chicago' in all_data_filename:
        graph_x = x[:, :, [0, 39, 40], :, :].reshape(
            (x.shape[0], x.shape[1], -1, north_south_map*west_east_map))
        high_graph_x = high_x[:, :, [0, 39, 40], :, :].reshape(
            (high_x.shape[0], high_x.shape[1], -1, north_south_map*west_east_map))
        graph_x = np.dot(graph_x, grid_node_map)
        high_graph_x = np.dot(high_graph_x, grid_node_map)

    print("feature:", str(x.shape), "label:", str(y.shape), "time:", str(target_times.shape),
            "high feature:", str(high_x.shape), "high label:", str(high_y.shape))
    print("graph_x:", str(graph_x.shape),
            "high_graph_x:", str(high_graph_x.shape))
    if x.shape[0] ==1080:
        b = np.ones_like(x)
        b = np.where(x>1,1,0)
        print('x的最大值:{}'.format(np.sum(b)))
        print('x的最小值:{}'.format(np.min(x)))
    if idx == 0:
        scaler = scaler
        train_data_shape = x.shape
        time_shape = target_times.shape
        graph_feature_shape = graph_x.shape
    loaders.append(Data.DataLoader(
        Data.TensorDataset(
            torch.from_numpy(x),
            torch.from_numpy(target_times),#4584*32
            torch.from_numpy(graph_x),
            torch.from_numpy(y)
        ),
        batch_size=batch_size,
        shuffle=(idx == 0)
    ))
    if idx == 2:
        high_test_loader = Data.DataLoader(
            Data.TensorDataset(
                torch.from_numpy(high_x),
                torch.from_numpy(high_target_times),
                torch.from_numpy(high_graph_x),
                torch.from_numpy(high_y)
            ),
            batch_size=batch_size,
            shuffle=(idx == 0)
        )
train_loader, val_loader, test_loader = loaders
################ 获取测试数据##################
nums_of_filter = []
for _ in range(2):
    nums_of_filter.append(gcn_num_filter)

GSNet_Model = GSNet(train_data_shape[2], num_of_gru_layers, seq_len, pre_len,
                    gru_hidden_size, time_shape[1], graph_feature_shape[2],
                    nums_of_filter, north_south_map, west_east_map)
print("模型参数---------------------------------------------")
print(train_data_shape[2], num_of_gru_layers, seq_len, pre_len,
        gru_hidden_size, time_shape[1], graph_feature_shape[2],
        nums_of_filter)

# multi gpu
if torch.cuda.device_count() > 10:
    print("Let's use", torch.cuda.device_count(), "GPUs!", flush=True)
    GSNet_Model = nn.DataParallel(GSNet_Model)
############### 载入模型######################
GSNet_Model.to(device)
# print(GSNet_Model)
GSNet_Model.load_state_dict(torch.load("data/model_parameter.pkl"))
print("模型加载成功！")

num_of_parameters = 0
for name, parameters in GSNet_Model.named_parameters():
    num_of_parameters += np.prod(parameters.shape)
print("Number of Parameters: {}".format(num_of_parameters), flush=True)

trainer = optim.Adam(GSNet_Model.parameters(), lr=learning_rate)
early_stop = EarlyStopping(patience=patience, delta=delta)

risk_mask = get_mask(mask_filename)
road_adj = get_adjacent(road_adj_filename)
risk_adj = get_adjacent(risk_adj_filename)
if poi_adj_filename == "":
    poi_adj = None
else:
    poi_adj = get_adjacent(poi_adj_filename)


feature: (4584, 7, 48, 20, 20) label: (4584, 1, 20, 20) time: (4584, 32) high feature: (1337, 7, 48, 20, 20) high label: (1337, 1, 20, 20)
graph_x: (4584, 7, 3, 243) high_graph_x: (1337, 7, 3, 243)
feature: (1080, 7, 48, 20, 20) label: (1080, 1, 20, 20) time: (1080, 32) high feature: (315, 7, 48, 20, 20) high label: (315, 1, 20, 20)
graph_x: (1080, 7, 3, 243) high_graph_x: (315, 7, 3, 243)
x的最大值:7
x的最小值:0.0
feature: (1080, 7, 48, 20, 20) label: (1080, 1, 20, 20) time: (1080, 32) high feature: (315, 7, 48, 20, 20) high label: (315, 1, 20, 20)
graph_x: (1080, 7, 3, 243) high_graph_x: (315, 7, 3, 243)
x的最大值:3
x的最小值:0.0
模型参数---------------------------------------------
48 5 7 1 256 32 3 [64, 64]
模型加载成功！
Number of Parameters: 6278578


In [7]:
#配置log日志输出
from myfunction import get_root_logger, logger_info
logger = get_root_logger('INFO', './logdata')
logger.info("Loading config file from ")


2023-05-02 01:00:26,517 - INFO - Loading config file from 


In [8]:
#torch.cuda.empty_cache()

In [10]:
#攻击，并写入数据,20230315导入包修改为lib.utils_new_round
import yaml
from mmcv import Config
from myfunction import log_test_results
from lib.Imperceptible_attack import log_test_csv,node_map,prenoise_pgd,rcr_pgd,fgsm,min,kl_pgd,JS_pgd_impr,rand_attack,nonenoise_pgd
from lib.Imperceptible_attack import ZINBSC,fgsm_impr,min_impr,pgd_impr,random_impr,log_test_csv_time_K,ZINBSC_geneater_kk
from myfunction import get_root_logger, logger_info
from lib.admm_atack_new import none_admm_attack
from lib.distru_attack import none_distri_attack
from lib.vers_attack import vers_attack
from lib.select import select_attack
#配置logo
logger = get_root_logger('INFO', './logdata')
logger.info("Loading config file from ")
cfgin = Config.fromfile('attack_setting.yaml')
#根据配置选择攻击方法
if cfgin.attacker == 'white_attacker':
    header = ['time','epoch','dataset', 'model','node_select', 'method', 'K','batch size', 
              'clean_RMSE', 'clean_recall', 'clean_MAP', 'clean_RCR',
              'adv_RMSE', 'adv_recall', 'adv_MAP', 'adv_RCR',
              'local_adv_RMSE', 'local_adv_recall', 'local_adv_MAP', 'local_adv_RCR',
              '备注信息：特征分析，选择攻击节点50，e，alldata = rand,其他不变']
    #文件名
    file_name = '第11a次-{}-eps{}-model-{}'.format(
        cfgin.dataset, cfgin.test_epsilon, cfgin.backbone)
    log_test_results(cfgin.result_dir, header, file_name)
    #可能的时间循环
    #攻击方法+节点选择组合
    for epoch in range(3):
        for out_K in cfgin.K:
            for i in cfgin.select_node:
                for j in cfgin.attack_method:
                    logger.info(i+'_'+j)
                    parm_att = j #攻击方法的函数名称
                    parm_node = i #节点选择的方法
                    #进行攻击,输出攻击日志
                    # 1. 节点选择，返回攻击的节点
                    ack_map = node_map(i,out_K, cfgin, GSNet_Model, test_loader, risk_mask, road_adj, risk_adj, poi_adj,
                                        grid_node_map, device, data_type='nyc')
                    # 2. 攻击方法选择，返回未攻击后的预测值，标签值,攻击前的预测值，攻击过程打印每个信息
                    adv_val_predict, val_target, val_predict = eval("{0}".format(parm_att))(logger, cfgin, ack_map, GSNet_Model, test_loader,  road_adj, risk_adj, poi_adj,
                                                                                            grid_node_map, scaler, risk_mask, device, data_type='nyc')
                    #计算结果并打印、存储,注意表头信息需要传递参数
                    log_test_csv_time_K(epoch,out_K,adv_val_predict, val_target,
                                         val_predict,cfgin,i,j,file_name,risk_mask)
                    torch.cuda.empty_cache()
           

2023-05-02 01:02:26,982 - INFO - Loading config file from 
2023-05-02 01:02:26,985 - INFO - Centrality_fgsm_impr


time,epoch,dataset,model,node_select,method,K,batch size,clean_RMSE,clean_recall,clean_MAP,clean_RCR,adv_RMSE,adv_recall,adv_MAP,adv_RCR,local_adv_RMSE,local_adv_recall,local_adv_MAP,local_adv_RCR,备注信息：特征分析，选择攻击节点50，e，alldata = rand,其他不变


2023-05-02 01:02:28,885 - INFO - Info:  [64/1080 (6%)]	  clean_RMSE: 7.1454 clean_Recall: 35.7388 clean_MAP: 0.1823 clean_RCR: 0.6776   adv_RMSE: 7.0075 adv_Recall: 36.0825 adv_MAP: 0.1838 adv_RCR: 0.6842 time:0.836
2023-05-02 01:02:30,551 - INFO - Info:  [128/1080 (12%)]	  clean_RMSE: 8.5350 clean_Recall: 38.0734 clean_MAP: 0.1698 clean_RCR: 0.6284   adv_RMSE: 8.3455 adv_Recall: 38.5321 adv_MAP: 0.1724 adv_RCR: 0.6282 time:0.826
2023-05-02 01:02:32,232 - INFO - Info:  [192/1080 (18%)]	  clean_RMSE: 8.2390 clean_Recall: 30.0781 clean_MAP: 0.1902 clean_RCR: 0.6865   adv_RMSE: 8.0533 adv_Recall: 30.4688 adv_MAP: 0.1951 adv_RCR: 0.6846 time:0.815
2023-05-02 01:02:33,912 - INFO - Info:  [256/1080 (24%)]	  clean_RMSE: 7.3895 clean_Recall: 40.7609 clean_MAP: 0.2031 clean_RCR: 0.6433   adv_RMSE: 7.2704 adv_Recall: 41.0326 adv_MAP: 0.2079 adv_RCR: 0.6483 time:0.819
2023-05-02 01:02:35,593 - INFO - Info:  [320/1080 (29%)]	  clean_RMSE: 7.9974 clean_Recall: 26.4550 clean_MAP: 0.1315 clean_RCR: 0

2023-05-02 01:02:57,0,NYC,Gsnet,Centrality,fgsm_impr,10,32,7.8062616694831215,33.24792581747194,0.178880014328625,0.6745602564431761,7.6441756971319705,33.296730112249875,0.17852347889785747,0.6763141027788449,0.16532887739135238,99.61505961881514,0.9952744670411922,0.0010287188135239148


2023-05-02 01:03:06,505 - INFO - Info:  [64/1080 (6%)]	  clean_RMSE: 7.1454 clean_Recall: 35.7388 clean_MAP: 0.1823 clean_RCR: 0.6776   adv_RMSE: 6.6664 adv_Recall: 33.3333 adv_MAP: 0.1694 adv_RCR: 0.7118 time:2.265
2023-05-02 01:03:11,213 - INFO - Info:  [128/1080 (12%)]	  clean_RMSE: 8.5350 clean_Recall: 38.0734 clean_MAP: 0.1698 clean_RCR: 0.6284   adv_RMSE: 7.8671 adv_Recall: 35.7798 adv_MAP: 0.1533 adv_RCR: 0.6573 time:2.313
2023-05-02 01:03:15,857 - INFO - Info:  [192/1080 (18%)]	  clean_RMSE: 8.2390 clean_Recall: 30.0781 clean_MAP: 0.1902 clean_RCR: 0.6865   adv_RMSE: 7.4720 adv_Recall: 30.0781 adv_MAP: 0.1966 adv_RCR: 0.6883 time:2.282
2023-05-02 01:03:20,501 - INFO - Info:  [256/1080 (24%)]	  clean_RMSE: 7.3895 clean_Recall: 40.7609 clean_MAP: 0.2031 clean_RCR: 0.6433   adv_RMSE: 6.8723 adv_Recall: 39.6739 adv_MAP: 0.1950 adv_RCR: 0.6577 time:2.330
2023-05-02 01:03:25,204 - INFO - Info:  [320/1080 (29%)]	  clean_RMSE: 7.9974 clean_Recall: 26.4550 clean_MAP: 0.1315 clean_RCR: 0

...

2023-05-02 12:17:20,2,NYC,Gsnet,SLT,pgd_impr,50,32,7.8062616694831215,33.24792581747194,0.178880014328625,0.6745602564431761,6.745070176038756,31.82040019521718,0.17448134726119505,0.6912584967389954,1.6770381171092035,98.8602009201014,0.9770102783874439,0.0072396483117849
(1080, 20, 20)


2023-05-02 12:17:46,836 - INFO - Info:  [64/1080 (6%)]	  clean_RMSE: 7.1454 clean_Recall: 35.7388 clean_MAP: 0.1823 clean_RCR: 0.6776   adv_RMSE: 6.8164 adv_Recall: 35.3952 adv_MAP: 0.1829 adv_RCR: 0.7078 time:0.763
2023-05-02 12:17:48,480 - INFO - Info:  [128/1080 (12%)]	  clean_RMSE: 8.5350 clean_Recall: 38.0734 clean_MAP: 0.1698 clean_RCR: 0.6284   adv_RMSE: 8.1722 adv_Recall: 36.6972 adv_MAP: 0.1681 adv_RCR: 0.6494 time:0.820
2023-05-02 12:17:50,072 - INFO - Info:  [192/1080 (18%)]	  clean_RMSE: 8.2390 clean_Recall: 30.0781 clean_MAP: 0.1902 clean_RCR: 0.6865   adv_RMSE: 7.8021 adv_Recall: 30.4688 adv_MAP: 0.1891 adv_RCR: 0.6812 time:0.766
2023-05-02 12:17:51,653 - INFO - Info:  [256/1080 (24%)]	  clean_RMSE: 7.3895 clean_Recall: 40.7609 clean_MAP: 0.2031 clean_RCR: 0.6433   adv_RMSE: 7.0174 adv_Recall: 41.0326 adv_MAP: 0.2053 adv_RCR: 0.6576 time:0.806
2023-05-02 12:17:53,286 - INFO - Info:  [320/1080 (29%)]	  clean_RMSE: 7.9974 clean_Recall: 26.4550 clean_MAP: 0.1315 clean_RCR: 0

2023-05-02 12:18:14,2,NYC,Gsnet,SLT,random_impr,50,32,7.8062616694831215,33.24792581747194,0.178880014328625,0.6745602564431761,7.405370990080944,33.23572474377745,0.18182804324957175,0.6746270753370217,0.3489237514365277,99.55121584827715,0.9944498670140476,0.001303467612542955
(1080, 20, 20)


2023-05-02 12:18:44,886 - INFO - Info:  [64/1080 (6%)]	  clean_RMSE: 7.1454 clean_Recall: 35.7388 clean_MAP: 0.1823 clean_RCR: 0.6776   adv_RMSE: 6.5423 adv_Recall: 33.6770 adv_MAP: 0.2020 adv_RCR: 0.7056 time:2.859
2023-05-02 12:18:50,781 - INFO - Info:  [128/1080 (12%)]	  clean_RMSE: 8.5350 clean_Recall: 38.0734 clean_MAP: 0.1698 clean_RCR: 0.6284   adv_RMSE: 7.8197 adv_Recall: 32.5688 adv_MAP: 0.1804 adv_RCR: 0.6870 time:2.868
2023-05-02 12:18:56,689 - INFO - Info:  [192/1080 (18%)]	  clean_RMSE: 8.2390 clean_Recall: 30.0781 clean_MAP: 0.1902 clean_RCR: 0.6865   adv_RMSE: 6.9859 adv_Recall: 32.4219 adv_MAP: 0.1868 adv_RCR: 0.6892 time:2.870
2023-05-02 12:19:02,604 - INFO - Info:  [256/1080 (24%)]	  clean_RMSE: 7.3895 clean_Recall: 40.7609 clean_MAP: 0.2031 clean_RCR: 0.6433   adv_RMSE: 6.3332 adv_Recall: 40.2174 adv_MAP: 0.2152 adv_RCR: 0.6638 time:2.872
2023-05-02 12:19:08,525 - INFO - Info:  [320/1080 (29%)]	  clean_RMSE: 7.9974 clean_Recall: 26.4550 clean_MAP: 0.1315 clean_RCR: 0

2023-05-02 12:20:27,2,NYC,Gsnet,SLT,ZINBSC,50,32,7.8062616694831215,33.24792581747194,0.178880014328625,0.6745602564431761,6.884478329555347,31.710590531966815,0.17209016228286694,0.6942968578535437,1.8908361521946875,98.8827340155854,0.9773411512408032,0.0068127174214604675
